# Mutation-Based Fuzzing

Most [randomly generated inputs](Basic_Fuzzing.ipynb) are syntactically _invalid_ and thus are quickly rejected by the processing program.  To exercise functionality beyond input processing, we must increase chances to obtain valid inputs.  One such way is by _mutating_ existing valid inputs - that is, introducing small changes that may still keep the input valid, yet exercise new behavior.

**Prerequisites**

* You should know how basic fuzzing works; for instance, from the ["Fuzzing"](Basic_Fuzzing.ipynb) chapter.

## The Problem

Most modern programs do a good job in _validating_ their inputs before they actually process them.  As an example, think of a _compiler_ translating program code into a lower-level language.  The processing steps of a compiler are typically depicted as a pipeline of components, each processing the input of its predecessors and producing output for its successors.  In the beginning, we typically have a lexical analysis that puts letters together into words, a syntactic analysis that puts sequences of words and items into structures, and then the actual compilation steps that translate these structures into code:

![Compiler Pipeline](PICS/Compiler.pdf)

The problem is that with random inputs, you will be able to exercise a lot of functionality in the leftmost stages (i.e., the lexical and possibly syntactical analyses), but the chances of actually producing a _valid_ input that will make it to the later stages are slim.


### Fuzzing Python Expressions

To illustrate just how low the chances are, let us _illustrate the problem on the Python interpreter._  Let us get the fuzzing function from  the ["Fuzzing"](Basic_Fuzzing.ipynb) chapter.

In [238]:
import gstbook
from Basic_Fuzzing import fuzzer

We can use the `fuzzer` function to generate random inputs.  Would this be a valid Python expression?

In [239]:
fuzzer()

'22 \' 0,4/&<3"#>)!?*%.(7<-4-" ,*?&#$)4*8.?9?5+:>3,>3/"-*6-53>0*4035(2;#$4:+&\':88-"\'!?07:$8'

To test which inputs are actually valid, we use the Python `parser` module.  `parser.suite(_source_)` returns an internal object (actually, a parse tree) if _source_ is a valid command:

In [240]:
import parser

parser.suite("print(2 + 2)")

Note that the command is _not_ executed.  (Actually, evaluating or otherwise executing randomly generated strings would be quite a risk: What happens if by chance, a string that deletes your files would be created?)

If _source_ is invalid, `parser.suite()` raises an exception:

In [241]:
from gstbook.expect_error import ExpectError

with ExpectError():
    parser.suite("print(2 +<>^& 37)")

Traceback (most recent call last):
  File "<ipython-input-241-a6d15798aa29>", line 4, in <module>
    parser.suite("print(2 +<>^& 37)")
  File "<string>", line 1
    print(2 +<>^& 37)
              ^
SyntaxError: invalid syntax


We can thus write a function `is_valid_expr()` that checks whether an expression is valid in Python:

In [242]:
def is_valid_expr(source):
    """Returns true iff source is a valid Python expression"""
    try:
        parser.suite("print(" + source + ")")
        return True
    except SyntaxError:
        return False
    except ValueError:
        return False

assert is_valid_expr("4 + 4")
assert not is_valid_expr("37 !@#$ 564")

### Invalid Inputs

Let us see how many of the `fuzzer()` outputs are actually valid:W

In [243]:
valid_inputs = set()
TRIALS = 1000

for i in range(TRIALS):
    input = fuzzer()
    if is_valid_expr(input):
        valid_inputs.add(input)
        
len(valid_inputs) / TRIALS

0.007

About 99% of all generated inputs are invalid - that's not very many.  What are the valid ones we get?

In [244]:
valid_inputs

{'',
 ')/9#+39?("%4!910(.;!\'5!0)7>>4*;98+"&#"0.,=&#\' >;355>,#,=<6!?59',
 '0',
 '1)#"1;?;"50;3,$?26:0>',
 '1);#-2:>#)&:3+2.;1/-$4> 4= ;6#8# #$)#!8$2!:3<(4?#\'$6+6."%>)9/0(29!,;4+(\'0<)$$16<7(".>4%>#4*$',
 '2',
 '5'}

While we do have a chance to create numbers and very simple arithmetic expressions, we are going to miss plenty of Python data types and functionality - and of course, we are not going to cover the code that handles these.  

What are the odds of producing a Python set, for instance? It would have to start with `set(`.  In its default configuration, for instance, `get_fuzzer()` does not even produce letters.  If we give it a range of say, 64 characters, we have a chance of $1 : 64^4$ to have an input that starts with `set(`.  How much is that, again?

In [245]:
64 ** 4

16777216

Indeed, less than one in a million.  Plus, we'd also need the closing `)`...

## Mutating Inputs

The alternative to generating random strings from scratch is to start with a guiven _valid_ input, and then to subsequently _mutate_ it.  A _mutation_ in this context is a simple string manipulation - say, inserting a (random) character, deleting a character, or flipping a bit in a character representation.  Here are some mutations to get you started:

In [246]:
import random

In [247]:
def delete_random_character(s):
    """Returns s with a random character deleted"""
    pos = random.randint(0, len(s) - 1)
    # print("Deleting", repr(s[pos]), "at", pos)
    return s[:pos] + s[pos + 1:]

for i in range(10):
    x = delete_random_character("A quick brown fox")
    print(x)

A quick bron fox
A quickbrown fox
A quick brow fox
A uick brown fox
A quck brown fox
A quic brown fox
A quick brow fox
A quickbrown fox
A quick brown fo
A quick rown fox


In [248]:
def insert_random_character(s):
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 64))
    # print("Inserting", repr(random_character), "at", pos)
    return s[:pos] + random_character + s[pos:]

for i in range(10):
    print(insert_random_character("A quick brown fox"))

A qui$ck brown fox
A quick brown /fox
A quick brown fox 
A quick brown 0fox
A quick 4brown fox
A quick brow>n fox
A quick brown fox(
A quick, brown fox
A quick brown =fox
A q8uick brown fox


In [249]:
def flip_random_character(s):
    """Returns s with a random bit flipped in a random position"""
    pos = random.randint(0, len(s) - 1)
    c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(c) ^ bit)
    # print("Flipping", bit, "in", repr(c) + ", giving", repr(new_c))
    return s[:pos] + new_c + s[pos + 1:]

for i in range(10):
    print(flip_random_character("A quick brown fox"))

A quick bzown fox
A quick brow~ fox
A quico brown fox
A quick browN fox
A0quick brown fox
A qtick brown fox
A quick crown fox
C quick brown fox
A qumck brown fox
A quick brosn fox


Let us now create a random mutator that randomly chooses which mutation to apply:

In [250]:
mutators = [delete_random_character, insert_random_character, flip_random_character]

def mutate(s):
    """Return s with a random mutation applied"""
    mutator = random.choice(mutators)
    # print(mutator)
    return mutator(s)


for i in range(10):
    print(mutate("A quick brown fox"))

A quick brow fox
A quic%k brown fox
A qu=ick brown fox
A quick brown fo3x
A quick broun fox
A quick brown &ox
A' quick brown fox
A qick brown fox
 A quick brown fox
A quickbrown fox


Let us now apply the `mutate()` function on a Python expression and see how many valid inputs we obtain.

In [251]:
seed_input = "1 + 2 * 3 / 4"
valid_inputs = set()
TRIALS = 1000

for i in range(TRIALS):
    input = mutate(seed_input)
    if is_valid_expr(input):
        valid_inputs.add(input)

The first thing we observe is that the number of valid inputs is now much higher.  We are still far away from 100% valid inputs, but this is an improvement:

In [252]:
len(valid_inputs) / TRIALS

0.096

Most important, though, is that the valid inputs now cover many more Python expression features - additional operands, identifiers, and more:

In [253]:
print(valid_inputs)

{'1 + 2 *  3 / 4', '1 + 2 * 3 / 0', '9 + 2 * 3 / 4', '1 + 82 * 3 / 4', '1 + r * 3 / 4', '1+ + 2 * 3 / 4', '0 + 2 * 3 / 4', '1 + 2 * 3 / 6', '1 + 2 * 32 / 4', '1 + 2 * 83 / 4', '1< + 2 * 3 / 4', '1 + .2 * 3 / 4', '1 + 2 * 39 / 4', '19 + 2 * 3 / 4', '1 + 3 * 3 / 4', '1 + 2 * 3 / 5', '-1 + 2 * 3 / 4', ' 1 + 2 * 3 / 4', '1 + 2 * 3 / 45', '1 + 2 * 3 /  4', '1 + 2 * 3/ 4', '1 + 2 * 30 / 4', '1 / 2 * 3 / 4', '1 + 2 * 53 / 4', '1 =+ 2 * 3 / 4', '1 + 2 * 2 / 4', '10+ 2 * 3 / 4', '1 + 26 * 3 / 4', '1 + 2 * 7 / 4', '1 + 2 * 36 / 4', '1 + 2 * 3 / 41', '1 + 0 * 3 / 4', '1 /+ 2 * 3 / 4', '1 + 2 * -3 / 4', '1 + 2 * s / 4', '1 + 2 * 35 / 4', '1 + 2 * 3 /4', '1 + 2 * 3. / 4', '1 + 2 *3 / 4', '1 + 2 * 1 / 4', '1 + 2 * 3  / 4', '1 + 2 + 3 / 4', '1 * 2 * 3 / 4', '1 + 2 * 33 / 4', '3 + 2 * 3 / 4', '1 + 20 * 3 / 4', '1 + 2 * 3 + 4', ' + 2 * 3 / 4', '1. + 2 * 3 / 4', '1 + 2  * 3 / 4', '10 + 2 * 3 / 4', '1 + 2 * 3 - 4', '1 + +2 * 3 / 4', '1 + 2 * 3 // 4', '1 + 2 * 63 / 4', 'q + 2 * 3 / 4', '1 + 72 * 3 / 4', '

## Next Steps

* Apply on files
* Make this guided by coverage, as AFL does